In [1]:
# Je ne sais pas vous, mais personnellement j'ai un écran large et ça m'agace
# d'avoir le notebook juste au milieu
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
# librairies mathématiques et de gestion
import pandas as pd
import pickle

# On charge les dataframes :
 - df, qui contient toutes les données incident et mobilisation
 - station_coord, qui contient les coordonnées des stations présentes dans les données

In [12]:
# j'ai passé les data en csv, avec un nom un peu moins pénible
# la lecture doit se faire avec latin-1 pour certain caractère spéciaux
df2 = pd.read_csv('data/full_incident.csv',sep=',', encoding='latin-1')
df2.info()

C:\Users\Roman\anaconda3\envs\ptorch\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (0,26,28,33,34) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309284 entries, 0 to 1309283
Data columns (total 38 columns):
 #   Column                                  Non-Null Count    Dtype  
---  ------                                  --------------    -----  
 0   IncidentNumber                          1309284 non-null  object 
 1   DateOfCall                              1309284 non-null  object 
 2   CalYear                                 1309284 non-null  int64  
 3   TimeOfCall                              1309284 non-null  object 
 4   HourOfCall                              1309284 non-null  int64  
 5   IncidentGroup                           1309283 non-null  object 
 6   StopCodeDescription                     1309283 non-null  object 
 7   SpecialServiceType                      399872 non-null   object 
 8   PropertyCategory                        1309283 non-null  object 
 9   PropertyType                            1309283 non-null  object 
 10  AddressQualifier              

# Il y a trop de colonnes pour pouvoir faire tourner des algorithmes normalement. On ne va garder que celles relatives à l'incident (ID, type), les dates/heures, temps de parcourt, et coordonnées de l'incident

In [15]:
to_keep = [0, 1, 2, 3, 4, 5, 7, 21, 22]
df2 = df2.iloc[:, to_keep]
df2.head()

,IncidentNumber,DateOfCall,CalYear,TimeOfCall,HourOfCall,IncidentGroup,SpecialServiceType,Easting_rounded,Northing_rounded
0,235138081,01-janv-09,2009,00:00:37,0,Special Service,RTC,528650,176850
1,1091,01-janv-09,2009,00:00:46,0,Special Service,Assist other agencies,530450,179050
2,2091,01-janv-09,2009,00:03:00,0,Fire,NaN,533750,194450
3,3091,01-janv-09,2009,00:04:27,0,Fire,NaN,507750,182850
4,5091,01-janv-09,2009,00:05:39,0,Fire,NaN,531050,185350


In [19]:
df = pd.read_csv('data/full_mobilisation.csv',sep=',', encoding='latin-1')
df.info()

C:\Users\Roman\anaconda3\envs\ptorch\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (0,4,6,8,11,13,19,24,25,27,29,31,34) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2032481 entries, 0 to 2032480
Data columns (total 36 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   IncidentNumber            object 
 1   ResourceMobilisationID    float64
 2   Resource_Code             object 
 3   PerformanceReporting      object 
 4   TimeMobilised             object 
 5   TimeMobilisedTimeZoneID   float64
 6   TimeMobile                object 
 7   TimeMobileTimeZoneID      float64
 8   TimeArrived               object 
 9   TimeArrivedTimeZoneID     float64
 10  AttendanceTimeSeconds     float64
 11  TimeLeft                  object 
 12  TimeLeftTimeZoneID        float64
 13  TimeReturned              object 
 14  TimeReturnedTimeZoneID    float64
 15  DeployedFromStation_Code  object 
 16  DeployedFromStation_Name  object 
 17  DeployedFromLocation      object 
 18  MobilisationOrder         float64
 19  PlusCode_code             object 
 20  PlusCode_Description    

# Même chose, on ne va garder que celles relatives à l'incident (ID, type), les dates/heures, temps de parcourt, et coordonnées de l'incident

In [20]:
to_keep = [0, 4, 6, 8, 11, 13, 16]
df = df.iloc[:, to_keep]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2032481 entries, 0 to 2032480
Data columns (total 7 columns):
 #   Column                    Dtype 
---  ------                    ----- 
 0   IncidentNumber            object
 1   TimeMobilised             object
 2   TimeMobile                object
 3   TimeArrived               object
 4   TimeLeft                  object
 5   TimeReturned              object
 6   DeployedFromStation_Name  object
dtypes: object(7)
memory usage: 108.5+ MB


# On peut maintenant réunir les données sur la base de l'identifiant de l'incident

In [22]:
df = df.merge(df2, on='IncidentNumber')
df.info()

# Au final, on nettoies les valeurs manquantes dans les colonnes essentielles à la problématique (id, heure de l'appel, heure d'arrivée, coordonnées)

In [27]:
df = df.dropna(subset=df.columns[[1, 2, 3, 6, 7, 8, 9, 10, 13, 14]])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1258146 entries, 1 to 1383935
Data columns (total 15 columns):
 #   Column                    Non-Null Count    Dtype 
---  ------                    --------------    ----- 
 0   IncidentNumber            1258146 non-null  object
 1   TimeMobilised             1258146 non-null  object
 2   TimeMobile                1258146 non-null  object
 3   TimeArrived               1258146 non-null  object
 4   TimeLeft                  1212722 non-null  object
 5   TimeReturned              1041477 non-null  object
 6   DeployedFromStation_Name  1258146 non-null  object
 7   DateOfCall                1258146 non-null  object
 8   CalYear                   1258146 non-null  int64 
 9   TimeOfCall                1258146 non-null  object
 10  HourOfCall                1258146 non-null  int64 
 11  IncidentGroup             1258146 non-null  object
 12  SpecialServiceType        291762 non-null   object
 13  Easting_rounded           1258146 non-null

# On peut enfin sauvegarder le dataset en csv

In [28]:
df.to_csv(r'data/Full_dataset.csv', index = False)